# **1) Import all packages**

In [350]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import math
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split 
from numpy.random import seed
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from scipy.special import logit
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler, Normalizer
from keras.models import Sequential
from keras.utils import to_categorical
from keras.regularizers import l1,l2
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

# **2) Prepare Data**

In [351]:
df = pd.read_csv('/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/UCI Breast Cancer Dataset/Data/data.csv')

In [352]:
df['diagnosis'].value_counts(normalize=True)

B    0.627417
M    0.372583
Name: diagnosis, dtype: float64

In [353]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [354]:
df = df.iloc[:,:-1]

In [355]:
print(df.head)

<bound method NDFrame.head of            id diagnosis  ...  symmetry_worst  fractal_dimension_worst
0      842302         M  ...          0.4601                  0.11890
1      842517         M  ...          0.2750                  0.08902
2    84300903         M  ...          0.3613                  0.08758
3    84348301         M  ...          0.6638                  0.17300
4    84358402         M  ...          0.2364                  0.07678
..        ...       ...  ...             ...                      ...
564    926424         M  ...          0.2060                  0.07115
565    926682         M  ...          0.2572                  0.06637
566    926954         M  ...          0.2218                  0.07820
567    927241         M  ...          0.4087                  0.12400
568     92751         B  ...          0.2871                  0.07039

[569 rows x 32 columns]>


In [356]:
#Split data
train, test =  train_test_split(df, train_size=0.85, test_size=0.15, random_state=42)

In [357]:
df = df.iloc[:,:-1]

In [358]:
train_id = train['id']
test_id = test['id']

In [359]:
train_data = train.iloc[:,1:]
test_data = test.iloc[:,1:]

In [360]:
#Scale data

# Training Data
train_x = train_data.iloc[:,1:]
train_x = MinMaxScaler().fit_transform(train_x)
print("Training Data :", train_x.shape)

# Testing Data
test_x = test_data.iloc[:,1:]
test_x = MinMaxScaler().fit_transform(test_x)
print("Testing Data :", test_x.shape)

Training Data : (483, 30)
Testing Data : (86, 30)


In [361]:
#Remove labels

# Training Data
train_y = train_data.iloc[:,:1]
train_y[train_y=='M'] = 0
train_y[train_y=='B'] = 1
print("Training Data :", train_y.shape)

# Testing Data
test_y = test_data.iloc[:,:1]
test_y[test_y=='M'] = 0
test_y[test_y=='B'] = 1
print("Testing Data :", test_y.shape)

Training Data : (483, 1)
Testing Data : (86, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [362]:
print(train_y)

    diagnosis
9           0
468         0
382         1
322         1
84          1
..        ...
71          1
106         1
270         1
435         0
102         1

[483 rows x 1 columns]


In [363]:
train_y =train_y['diagnosis'].to_numpy() 

In [364]:
train_y = train_y.astype(float)

# **3) Baseline Model**

In [365]:
test_y =test_y['diagnosis'].to_numpy() 
test_y = test_y.astype(float)

In [366]:
#Create Logistic Regression Model
logreg = LogisticRegression(max_iter= 3000)
logreg.fit(train_x,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [367]:
#Get AUC score
logit_roc_auc = roc_auc_score(test_y, logreg.predict(test_x))
print(logit_roc_auc)

0.9537037037037037


# **4) Vanilla Decision Tree**

In [446]:
#Create decision tree for specified depth of 5 and 10 
model = DecisionTreeClassifier(max_depth=5,random_state=16)
model.fit(train_x,train_y)
#max_depth=5,

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=16, splitter='best')

In [447]:
res_pred = model.predict(test_x)

In [444]:
print(test_y)

[1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.]


In [440]:
print(res_pred)

[1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.]


In [448]:
#Get various scores
acc = accuracy_score(res_pred,test_y)
rec = recall_score(test_y, res_pred,average='macro')
prec = precision_score(test_y, res_pred,average='macro')
auc = roc_auc_score(test_y, res_pred,average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.8604651162790697
Recall:  0.8825231481481481
Precision: 0.8576839826839827
AUC:  0.8825231481481481


# **5) Neural Network to extract logits**

In [373]:
#Prepare labels for Neural Network
y_train = np.array(keras.utils.to_categorical(train_y))
y_test = np.array(keras.utils.to_categorical(test_y))
print(y_test.shape)

(86, 2)


In [374]:
#Metrics for Neural Network
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def auc_m(y_true, y_pred):
     auc = tf.metrics.auc(y_true, y_pred)[1]
     K.get_session().run(tf.local_variables_initializer())
     return auc

In [375]:
#Built Neural Network
def build_model():
  inputs = layers.Input(shape=(30))
  x = layers.Dense(10,activation='relu',name='fc1')(inputs)
  
  x = layers.Dropout(rate=0.5, name='Dropout1')(x)

  x = layers.Dense(10,activation='relu',name='fc2')(x)

  x = layers.Dense(5,activation='relu',name='fc3')(x)

  x = layers.Dense(2,name='logits')(x)

  preds = layers.Activation('softmax',name='Softmax')(x)

  model = Model(inputs=inputs, outputs=preds)
  model.summary()
  return model
model = build_model()

Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 10)                310       
_________________________________________________________________
Dropout1 (Dropout)           (None, 10)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                110       
_________________________________________________________________
fc3 (Dense)                  (None, 5)                 55        
_________________________________________________________________
logits (Dense)               (None, 2)                 12        
_________________________________________________________________
Softmax (Activation)         (None, 2)               

In [376]:
#Set-up filepath and ModelCheckepoint with Earlystopping to save best performing moddel based on Validation AUC
filepath = "/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/UCI Breast Cancer Dataset/UCI Saved Models/UCI_MODELV6.hdf5"
model_checkpoint = ModelCheckpoint(filepath,  monitor='val_auc', save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_auc', mode='max', verbose=0, patience=5)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC(name='auc')])

In [377]:
#Set epochs and bach size, and train model 
num_epoch = 100
batch_size = 64
model1 = model.fit(x=train_x, y=y_train, epochs=num_epoch, batch_size=batch_size, validation_split=0.15,callbacks = [model_checkpoint,es])

Epoch 1/100
7/7 [==============================] - 1s 94ms/step - loss: 0.7001 - acc: 0.4171 - f1_m: 0.4181 - precision_m: 0.4186 - recall_m: 0.4176 - auc: 0.4526 - val_loss: 0.6905 - val_acc: 0.3562 - val_f1_m: 0.3464 - val_precision_m: 0.3464 - val_recall_m: 0.3464 - val_auc: 0.5403
Epoch 2/100
7/7 [==============================] - 0s 10ms/step - loss: 0.6862 - acc: 0.4561 - f1_m: 0.4696 - precision_m: 0.4696 - recall_m: 0.4696 - auc: 0.5226 - val_loss: 0.6833 - val_acc: 0.4521 - val_f1_m: 0.4010 - val_precision_m: 0.4010 - val_recall_m: 0.4010 - val_auc: 0.5752
Epoch 3/100
7/7 [==============================] - 0s 10ms/step - loss: 0.6792 - acc: 0.5220 - f1_m: 0.5201 - precision_m: 0.5201 - recall_m: 0.5201 - auc: 0.5837 - val_loss: 0.6763 - val_acc: 0.6027 - val_f1_m: 0.4870 - val_precision_m: 0.4870 - val_recall_m: 0.4870 - val_auc: 0.6737
Epoch 4/100
7/7 [==============================] - 0s 9ms/step - loss: 0.6734 - acc: 0.6195 - f1_m: 0.6355 - precision_m: 0.6355 - recall_m: 0

In [378]:
#Train performance
prediction_model = tf.keras.models.load_model(filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(train_x,y_train, verbose=0, batch_size=32)

print('Train Accuracy:',accuracy)
print('Train Recall:',recall)
print('Train Precision:',precision)
print('Train F1:',f1_score)
print('Train AUC:',auc)
prediction_model.summary()

Train Accuracy: 0.9730848670005798
Train Recall: 0.974609375
Train Precision: 0.974609375
Train F1: 0.974609375
Train AUC: 0.9955912828445435
Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 10)                310       
_________________________________________________________________
Dropout1 (Dropout)           (None, 10)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                110       
_________________________________________________________________
fc3 (Dense)                  (None, 5)                 55        
_________________________________________________________________
logits (Dense)               (None, 2)     

In [379]:
#Test performance
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(test_x, y_test, verbose=0, batch_size=32)

print('Test Accuracy:',accuracy)
print('Test Recall:',recall)
print('Test Precision:',precision)
print('Test F1:',f1_score)
print('Test AUC:',auc)
prediction_model.summary()

Test Accuracy: 0.9418604373931885
Test Recall: 0.9431818127632141
Test Precision: 0.9431818127632141
Test F1: 0.9431817531585693
Test AUC: 0.9931043982505798
Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 10)                310       
_________________________________________________________________
Dropout1 (Dropout)           (None, 10)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                110       
_________________________________________________________________
fc3 (Dense)                  (None, 5)                 55        
_________________________________________________________________
logits (Dense)             

In [390]:
#Best Performance
best_filepath =  "/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/UCI Breast Cancer Dataset/UCI Saved Models/UCI_MODELV4.hdf5"
best_model = tf.keras.models.load_model(best_filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = best_model.evaluate(test_x,y_test, verbose=0, batch_size=16)

#print('Best Accuracy:',accuracy)
#print('Best Recall:',recall)
#print('Best Precision:',precision)
#print('Best F1:',f1_score)
print('Best AUC:',auc)
best_model.summary()

Best AUC: 0.9906030297279358
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 10)                310       
_________________________________________________________________
Dropout1 (Dropout)           (None, 10)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                110       
_________________________________________________________________
fc3 (Dense)                  (None, 5)                 55        
_________________________________________________________________
logits (Dense)               (None, 2)                 12        
_________________________________________________________________
Softmax (Activation)     

# **6) Extract Logits**

In [391]:
#Load the best model
#filepath =  "/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/UCI Breast Cancer Dataset/UCI Saved Models/UCI_MODELV4.hdf5"
model = tf.keras.models.load_model(best_filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m}) 

In [392]:
#Compile it again
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC()])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 10)                310       
_________________________________________________________________
Dropout1 (Dropout)           (None, 10)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                110       
_________________________________________________________________
fc3 (Dense)                  (None, 5)                 55        
_________________________________________________________________
logits (Dense)               (None, 2)                 12        
_________________________________________________________________
Softmax (Activation)         (None, 2)                

In [393]:
#Remove last softmax layer and predict again
model_sans_softmax = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = model_sans_softmax.predict(train_x)

In [394]:
#Get the logits
soft_targets = model_logits

# **7) Get Student Decision Tree**


In [449]:
#Set up Decision Tree for Knowledge Distillation at given depth 
model = DecisionTreeRegressor(max_depth=5)

In [450]:
#Train Decision Tree
model.fit(train_x,soft_targets)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [451]:
#Get predictions
res_pred = model.predict(test_x)

In [452]:
#Covnvert continious predictions to categorical ones
pred_argmax = np.zeros_like(res_pred)
pred_argmax[np.arange(len(res_pred)), res_pred.argmax(1)] = 1

In [453]:
#Get scores of student decision tree
acc = accuracy_score(pred_argmax,y_test)
rec = recall_score(y_test, pred_argmax,average='macro')
prec = precision_score(y_test, pred_argmax,average='macro')
auc = roc_auc_score(y_test, pred_argmax,average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.9069767441860465
Recall:  0.9131944444444444
Precision: 0.8966666666666667
AUC:  0.9131944444444444
